In [0]:
dbutils.fs.mount(source = 'wasbs://sales@trendytechsaproject.blob.core.windows.net', mount_point = '/mnt/sales',
extra_configs = {'fs.azure.account.key.trendytechsaproject.blob.core.windows.net': 'Z5+E5DIhGSQeFUWzv8pHoYaWEy+V+NimF7oGG9EKJziRX36InbYLvfBBz9+aE/SJ0jZG7VYsiJY9+ASty6biOA=='})

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
<command-6064040128614978> in <cell line: 1>()
----> 1 dbutils.fs.mount(source = 'wasbs://sales@trendytechsaproject.blob.core.windows.net', mount_point = '/mnt/sales',
      2 extra_configs = {'fs.azure.account.key.trendytechsaproject.blob.core.windows.net': 'Z5+E5DIhGSQeFUWzv8pHoYaWEy+V+NimF7oGG9EKJziRX36InbYLvfBBz9+aE/SJ0jZG7VYsiJY9+ASty6biOA=='})

/databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling(*args, **kwargs)
    360                     exc.__context__ = None
    361                     exc.__cause__ = None
--> 362                     raise exc
    363 
    364             return f_with_exception_handling

ExecutionError: An error occurred while calling o495.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/sales; nested exception is

In [0]:
%fs
ls  /mnt/sales

path,name,size,modificationTime
dbfs:/mnt/sales/discarded/,discarded/,0,0
dbfs:/mnt/sales/landing/,landing/,0,0
dbfs:/mnt/sales/staging/,staging/,0,0


In [0]:
%fs 
ls /mnt/sales/landing

path,name,size,modificationTime
dbfs:/mnt/sales/landing/orders.csv,orders.csv,258,1761161823000


In [0]:
ordersDF = spark.read.csv('/mnt/sales/landing/orders.csv', inferSchema= True, header = True)

In [0]:
display(ordersDF)

order_id,order_date,customer_id,order_status
1111111,2013-07-25T00:00:00.000+0000,11599,CLOSED
2222222,2013-07-25T00:00:00.000+0000,256,PENDING_PAYMENT
3333333,2013-07-25T00:00:00.000+0000,12111,COMPLETE
4444444,2013-07-25T00:00:00.000+0000,8827,CLOSED


In [0]:
errorFlg = False
ordersCount = ordersDF.count()
print(ordersCount)

distinctordersCount = ordersDF.select('order_id').distinct().count()
print(distinctordersCount)

if ordersCount != distinctordersCount:
    errorFlg = True
if errorFlg:
    dbutils.fs.mv('/mnt/sales/landing/orders.csv', '/mnt/sales/discarded')
    dbutils.notebook.exit('{"errorFlg": "true", "errorMsg": "Orderid is repeated"}')
else:
    ordersDF.createOrReplaceTempView("orders")

4
4


In [0]:
dbServer:'trendytechsqlserverprojectt'
dbPort :1433
dbName : 'trendytechsqldbproject'
dbUser : 'ttsqluser'
dbPassword : 'Pooja1234'
databricksScope : 'saleprojectscopenew'


In [0]:

connectionUrl = 'jdbc:sqlserver://{}.database.windows.net:{}; database={}; user={};'.format(dbServer, dbPort, dbName, dbUser)


connectionProperties = {
      'password' : 'Pooja1234',
      'driver': 'com.microsoft.sqlserver.jdbc.SQLServerDriver'
}

In [0]:
ValidstatusDF = spark.read.jdbc(url = connectionUrl, table = 'dbo.valid_order_status', properties = connectionProperties)

In [0]:
dbutils.secrets.listScopes()

Out[13]: [SecretScope(name='salesprojectsccope')]

In [0]:
display(ValidstatusDF)

status_name
ON_HOLD
PAYMENT_REVIEW
PROCESSING
CLOSED
SUSPECTED_FRAUD
COMPLETE
PENDING
CANCELLED
PENDING_PAYMENT


In [0]:
ValidstatusDF.createOrReplaceTempView("valid_status")

In [0]:
invalid_rows_df = spark.sql("select * from orders where order_status not in (select * from valid_status)")
display(invalid_rows_df)

order_id,order_date,customer_id,order_status


In [0]:
if invalid_rows_df.count() >0:
    errorFlg = True
if errorFlg:
    dbutils.fs.mv('/mnt/sales/landing/orders.csv','/mnt/sales/discarded')
    dbutils.notebook.exit('{"errorFlg": "true", "errorMsg": "Invalid order status found"}')
else:
     dbutils.fs.mv('/mnt/sales/landing/orders.csv','/mnt/sales/staging')
     dbutils.notebook.exit('{"errorFlg": "true", "errorMsg": "All good"}')